In [22]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib as plot
import math
import talib
from sklearn import metrics, linear_model, preprocessing, model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

In [23]:
conn = sqlite3.connect('poloniex_0.1.db')
c = conn.cursor()

In [24]:
df = pd.read_sql_query("SELECT * FROM candles_USDT_BTC", conn)
df = df.drop('id' ,1)
df = df.drop('trades' ,1)
df['start'] = pd.to_datetime(df['start'],unit='s')
df.set_index('start' , inplace = True)
#df.index.dtype
df.head(10)
#print (len(df.index))


,open,high,low,close,vwp,volume
start,,,,,,
2017-06-11 19:51:00,2923.996734,2932.600000,2923.996734,2925.828930,2929.711168,10.337254
2017-06-11 19:52:00,2930.373888,2932.600000,2925.828930,2925.996734,2928.910755,2.599433
2017-06-11 19:53:00,2932.600000,2932.944889,2925.996734,2930.000000,2930.652682,7.102010
2017-06-11 19:54:00,2932.944889,2935.107505,2923.494110,2923.996834,2930.371417,9.232048
2017-06-11 19:55:00,2923.996834,2935.000000,2923.996734,2923.996734,2928.012814,1.390456
2017-06-11 19:56:00,2932.835523,2932.835523,2923.996734,2923.996734,2924.764786,3.015836
2017-06-11 19:57:00,2923.996735,2930.528568,2923.996735,2930.528568,2924.291140,1.462843
2017-06-11 19:58:00,2923.996735,2932.835522,2923.996735,2930.548878,2931.113139,6.586996
2017-06-11 19:59:00,2930.548878,2932.835522,2924.416129,2928.274152,2927.959137,0.607046


In [25]:
def safediv(x,y):
    if y == 0 :
        return np.nan
    else:
        return x/y

In [26]:
def ohlcsum(df):
    #df = df.sort()
    
    return {
       'open': df['open'][0],
       'high': df['high'].max(),
       'low': df['low'].min(),
       'close': df['close'][-1],
       'volume': df['volume'].sum(),
        
        'vwp': safediv((df['vwp']* df['volume']).sum(),df['volume'].sum()) 
        
      }

In [27]:
def convertcandles(df, mins):
    s = str(mins) + 'T'
    return( df.resample(s).agg(ohlcsum))
    
    
#convertcandles(df, 120)
    

In [28]:
newdf = convertcandles(df,30)

#newdf = df
price = np.array(newdf['vwp'])
h = np.array(newdf['high'])
l = np.array(newdf['low'])
c = np.array(newdf['close'])
#sma  = talib.SMA(price,50)
m = talib.MACD(price, 12,26,9)

mom = talib.MOM(price,12 )
rsi = talib.RSI(price)
krsi, drsi = talib.STOCHRSI(price)
roc = talib.ROC(price)
mean5d = pd.rolling_mean(newdf['vwp'],5)
mean10d = pd.rolling_mean(newdf['vwp'],10)
newdf['willR'] = talib.WILLR(h,l,c)
newdf['krsi'] = krsi
newdf['drsi'] = drsi
newdf['disp5d'] = newdf['vwp'] - mean5d
newdf['disp10d'] = newdf['vwp'] - mean10d
newdf['cci'] = talib.CCI(h,l,c)
newdf['roc'] = roc
#m[1][40:60]
# newdf['msig'] = np.where((m[1]>m[2]),1,0)
newdf['msig'] = m[1]-m[2]
#newdf['mom'] = np.where(mom>0,1,0)
# newdf['rsi'] = np.where((rsi>50)& (rsi> np.roll(rsi,1)),1,0)
newdf['rsi'] = rsi
# newdf['sma'] = np.where((pd.rolling_mean(newdf['vwp'],9)- pd.rolling_mean(newdf['vwp'],26))>0,1,0)
newdf['target'] = np.where((newdf['close'].shift(1)- newdf['close'])>0,1,0)
newdf.dropna(axis = 0, how = 'any', inplace = True)
newdf.drop(newdf.index[0], inplace = True)
newdf.head(30)


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:15: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=5,center=False).mean()
  from ipykernel import kernelapp as app
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:16: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=10,center=False).mean()
  app.launch_new_instance()


,open,high,low,close,vwp,volume,willR,krsi,drsi,disp5d,disp10d,cci,roc,msig,rsi,target
start,,,,,,,,,,,,,,,,
2017-06-12 12:30:00,2690.000000,2735.000000,2650.000000,2681.224827,2694.583479,1006.150065,-58.123426,100.000000,6.778397e+01,43.488929,-46.672959,-63.199488,-6.395221,-20.133604,29.072706,1
2017-06-12 13:00:00,2681.224827,2687.972300,2625.298278,2672.934000,2654.024950,752.270442,-60.276887,84.233489,9.474450e+01,6.014911,-65.927494,-68.374785,-7.430594,-29.492888,25.532636,1
2017-06-12 13:30:00,2672.934000,2716.000000,2658.000000,2716.000000,2695.965940,553.904538,-47.806779,100.000000,9.474450e+01,31.896260,-9.012484,-37.599398,-5.261970,-40.152238,34.424796,0
2017-06-12 14:00:00,2716.000000,2730.000000,2688.000000,2695.000000,2711.137373,636.116292,-53.379170,100.000000,9.474450e+01,28.226551,17.891259,-25.516505,-4.147950,-49.025725,37.339585,1
2017-06-12 14:30:00,2695.000000,2700.000000,2658.348325,2670.000000,2676.795863,424.359693,-60.039290,69.082934,8.969431e+01,-9.705658,-1.613135,-40.421383,-5.251766,-53.591065,33.689223,1
2017-06-12 15:00:00,2670.000000,2670.000000,2593.539247,2608.337943,2622.494604,774.337962,-74.853943,28.361732,6.581489e+01,-49.589142,-39.094544,-81.387750,-6.027123,-54.687234,28.881291,1
2017-06-12 15:30:00,2608.337943,2657.950631,2560.000000,2648.000000,2620.668898,875.799142,-61.599420,0.000000,3.248156e+01,-44.743638,-36.042389,-79.545101,-1.827292,-56.804883,28.732817,0
2017-06-12 16:00:00,2648.000000,2666.875000,2580.000000,2619.000000,2623.282273,821.384232,-69.724776,6.510457,1.162406e+01,-27.593529,-34.190468,-74.435588,0.291113,-59.563892,29.293157,1
2017-06-12 16:30:00,2619.000000,2635.000000,2505.000000,2555.000000,2547.943684,1295.361545,-85.380117,0.000000,2.170152e+00,-70.293381,-102.630259,-146.075150,-2.636216,-58.671774,23.545719,1


In [29]:
factors = ['msig','rsi','cci','willR','krsi','drsi','disp5d','disp10d']
X = newdf[factors]
Y = newdf['target']

In [37]:
Xtrain,  Xtest, Ytrain, Ytest = model_selection.train_test_split(X,Y)
#Xtrain.head()
scaler = preprocessing.MinMaxScaler(copy = False)
scaler.fit(Xtrain)
Xtrain = scaler.transform(Xtest)
# mod = linear_model.LogisticRegression(solver = "saga")
# mod.fit(Xtrain, Ytrain)
scaler_filename = "scaler.save"
joblib.dump(scaler, scaler_filename) 

Xtest =scaler.transform(Xtest)
# print('LogisticRegression score: %f'
#       % mod.score(Xtest, Ytest))

In [31]:
# # names = [ "Linear SVM", "RBF SVM", "Gaussian Process",
# #          "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
# #          "Naive Bayes"]
# names = [ "Linear SVM", "RBF SVM", 
#          "Decision Tree", "Random Forest", "Neural Net", "AdaBoost","LogisticRegression"
#          ]
# # classifiers = [SVC(kernel="linear", C=0.025),
# #     SVC(gamma=2, C=1),
# #     GaussianProcessClassifier(1.0 * RBF(1.0), warm_start=True),
# #     DecisionTreeClassifier(max_depth=5),
# #     RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
# #     MLPClassifier(alpha=1),
# #     AdaBoostClassifier(),
# #     GaussianNB()]
# classifiers = [SVC(kernel="linear", C=0.025,probability=True),
#     SVC(gamma=2, C=1,probability=True),
#     DecisionTreeClassifier(max_depth=5),
#     RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
#     MLPClassifier(alpha=1),
#     AdaBoostClassifier(),LogisticRegression(solver = "saga")]
# for name, clf in zip(names, classifiers):
#     clf.fit(Xtrain, Ytrain)
#     clf_probs = clf.predict_proba(Xtest)
#     logscore = metrics.log_loss(Ytest, clf_probs)
#     score = clf.score(Xtest, Ytest)
#     print(name, score, logscore)

In [32]:
clf  = model_selection.GridSearchCV(SVC(probability=True),cv = 5, param_grid= {'C': np.logspace(-3, 2, 6), 'gamma': np.logspace(-3, 2, 6)})
clf.fit(Xtrain, Ytrain)
# clf_probs = clf.predict_proba(Xtest)
# logscore = metrics.log_loss(Ytest, clf_probs)
# score = clf.score(Xtest, Ytest)
#print( model.best_estimator_ )
print( clf.best_score_ )
print( clf.best_params_ )
# print(score, logscore)

0.764769647696
{'C': 100.0, 'gamma': 1.0}


In [35]:
from sklearn.externals import joblib
clf = SVC(gamma=1, C=100,probability=True)
clf.fit(Xtrain, Ytrain)
filename = 'finalized_model.sav'
joblib.dump(clf, filename)

['finalized_model.sav']